In [1]:
import uproot
import numpy as np
root_file = uproot.open("result_5384_130_6501.root")
root_file.allkeys()
TDC = root_file['TDC']
TC = root_file['TC']
# print(TC.allkeys())

def _minmax_from_branch(tree, bname, entry=0, offset=0):
    '''
    Awkward Array
    TDC['wire_index_u'].array()[event#][blob#]
    '''
    akarray = tree[bname].array()[entry]
    min = np.array([np.min(l)+offset for l in akarray])
    min = np.expand_dims(min, axis=1)
    max = np.array([np.max(l)+offset for l in akarray])
    max = np.expand_dims(max, axis=1)
    return np.concatenate((min,max), axis=1)

def _wire_charge_sum(tree, bname, entry=0):
    wire_charge = tree[bname].array()[entry]
    wire_charge = np.array([int(sum(l)) for l in wire_charge])
    wire_charge = np.expand_dims(wire_charge, axis=1)
    return wire_charge

def _signature(tree, entry=0):
    time_slice_mm = _minmax_from_branch(tree, 'time_slice',entry)
    time_slice_mm = time_slice_mm*4
    time_slice_mm[:,1] = time_slice_mm[:,1]+4
    wire_index_u_mm = _minmax_from_branch(tree, 'wire_index_u',entry,0)
    wire_index_v_mm = _minmax_from_branch(tree, 'wire_index_v',entry,2400)
    wire_index_w_mm = _minmax_from_branch(tree, 'wire_index_w',entry,4800)
    sig = np.concatenate((time_slice_mm,wire_index_u_mm,wire_index_v_mm,wire_index_w_mm), axis=1)
    wire_charge_u = _wire_charge_sum(tree,'wire_charge_u',entry)
    wire_charge_v = _wire_charge_sum(tree,'wire_charge_v',entry)
    wire_charge_w = _wire_charge_sum(tree,'wire_charge_w',entry)
    sig = np.concatenate((sig,wire_charge_u,wire_charge_v,wire_charge_w), axis=1)
    return sig
#     flag_u = np.array(tree['flag_u'].array()[entry])
#     flag_u = np.expand_dims(flag_u, axis=1)
#     flag_v = np.array(tree['flag_v'].array()[entry])
#     flag_v = np.expand_dims(flag_v, axis=1)
#     flag_w = np.array(tree['flag_w'].array()[entry])
#     flag_w = np.expand_dims(flag_w, axis=1)
#     sig = np.concatenate((sig,wire_charge_u,wire_charge_v,wire_charge_w,flag_u,flag_v,flag_w), axis=1)
#     return sig

def _sort(arr):
    ind = np.lexsort((arr[:,7],arr[:,6],arr[:,5],arr[:,4],arr[:,3],arr[:,2],arr[:,1],arr[:,0]))
    arr = np.array([arr[i] for i in ind])
    return arr

sigs = _signature(TC, 0)

sigs = _sort(sigs)
# sigs = sigs[sigs[:,11]==0,:]
sigs = sigs[sigs[:,0]<40,:]
print(sigs.shape)
for i in range(min([sigs.shape[0], 20])):
# for i in range(sigs.shape[0]):
    print(i, sigs[i,:])

(36, 11)
0 [    0     4  1517  1518  3560  3562  6805  6808   916  6852 37007]
1 [   0    4 1518 1518 3558 3558 6805 6805  494 1623 1033]
2 [    0     4  1521  1522  3558  3558  6807  6808  2983  1623 27511]
3 [    0     4  2073  2074  4329  4330  8130  8131  1667  3270 14744]
4 [    4     8  1517  1522  3558  3562  6805  6808 14728 21845 41226]
5 [    4     8  2071  2074  4329  4331  8129  8131 10007 11158 17357]
6 [    8    12  1518  1521  3558  3561  6805  6808 21384 23932 36489]
7 [   8   12 2067 2067 4333 4333 8129 8129  252  616 1749]
8 [   8   12 2070 2070 4333 4333 8131 8131 1576  616 7365]
9 [    8    12  2070  2073  4330  4331  8129  8131 13775 16067 20396]
10 [   12    16  1518  1520  3558  3561  6804  6808 29552 32984 37981]
11 [   12    16  2067  2067  4333  4334  8129  8130   745  1642 17116]
12 [   12    16  2069  2072  4330  4334  8129  8131 16890 20841 21570]
13 [   16    20  1515  1520  3558  3560  6804  6808 29745 41462 42124]
14 [   16    20  2068  2071  4331  4334 

In [25]:
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib tk
plt.plot(mm[:,2]*1e7+mm[:,3]*1e6+mm[:,4]*1e5,'o')
plt.grid()
plt.show()

In [26]:
l = np.array([[3,0.1,0.02],[1,0.2,0.03],[2,0.3,0.01],[2,0.1,0.03]])
l = l[l[:,2].argsort()]
print(l)
l = l[l[:,1].argsort()]
print(l)
l = l[l[:,0].argsort()]
print(l)

[[2.   0.3  0.01]
 [3.   0.1  0.02]
 [1.   0.2  0.03]
 [2.   0.1  0.03]]
[[3.   0.1  0.02]
 [2.   0.1  0.03]
 [1.   0.2  0.03]
 [2.   0.3  0.01]]
[[1.   0.2  0.03]
 [2.   0.1  0.03]
 [2.   0.3  0.01]
 [3.   0.1  0.02]]


In [60]:
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

# [Y, Z]
line_5 = [[1174.5, 35.49],[1154.21,0.35]]
line_6 = [[1174.5, 41.49],[1150.75,0.35]]
line_3078 = [[1174.45, 38.59], [-1155.05, 4073.41]]
# BB: Y < 1174.70
print(line_intersection(line_5, line_3078))
print(line_intersection(line_6, line_3078))

(1175.3699330244774, 36.99662624347637)
(1173.6378826284347, 39.99663121405449)


In [80]:
tree = TC
wire_charge_u = tree['wire_charge_u'].array()[0]
np.array([sum(l) for l in wire_charge_u])
sum(wire_charge_u[2])

2983.0

In [94]:
int(0.9)

0